# OFFBOARD Mode and Open Loop Control

At the beginning of this class we flew our drones in [`MANUAL`](https://docs.px4.io/en/flight_modes/manual_stabilized_mc.html) mode, piloting it using the remote control. This mode was difficult to control and required constant input from the pilot in order to prevent it from drifting off and crashing into the walls. This constant input from the pilot to maintain control was a required because the drone had very little knowledge of the world; i.e. it had no way of _localizing_ itself in the world which meant that it had no way of knowing that it's position was drifting toward a wall.

After implementing off-the-shelf [_optical flow_](enable_optical_flow.html) and [_ARTag localization_](ar_localization.html) libraries, we were able to fly the drone in [`POSITION`](https://docs.px4.io/en/flight_modes/position_mc.html)mode (sometimes refered to as `POSITION CONTROL` or `POSCTL`). This mode still took inputs from the pilot via the remote control, but it was _much_ easier to fly since the drone could effectively hover in place without drifting. In this mode, the input from the pilot commanded the drone to make slow movements in body-frame directions.

Our next step is to utilize the [`OFFBOARD`](https://docs.px4.io/en/flight_modes/offboard.html) mode, which is removes the need for pilot input and is our first venture into truly autonomous flight! This practical will walk us through sending short duration velocity commands that the drone will execute. We call it "open loop" because the commands are pre-scripted and don't adapt or update based on new information (i.e. sensor/camera data).

For more information on PX'4 various flight modes, [see here](https://docs.px4.io/en/flight_modes/)

<div class="alert alert-warning">

**WARNING:** 

`OFFBOARD` mode is potentially one of the more dangerous modes because you are handing over control of your quadrotor to computer program that may not be perfect. We will implement as many software-based safety checks as possible, but the last line of defence is still the [Pilot in Command](team_roles.html#Pilot-in-Command). Therefore, for safety purposes, the pilot we will __always have the remote control in hand in order to regain control when necessary__.

</div>

The learning objectives for this practical include:

1. Understand how to safely enable and operate PX4's `OFFBOARD` mode
2. Develop the code necessary to send velocity commands
3. Gain intuition about the various coordinate frames in which we can command velocities (e.g. world reference frames vs. body reference frames)


## Step 1: Complete Control Script

In the `aero_control` repository/ROS package, we have given you a incomplete script for running a simple open-loop translation control demonstration. The purpose of this script is to publish low-speed velocity commands in the form of ROS `Twist` messages to the topic `/mavros/setpoint_velocity/cmd_vel_unstamped` for a short duration (e.g. 3 seconds) and then send "hover" messages (i.e. `Twist` message of all zeros) causing the drone to safely hover in place. The script should also be able to take various coordinate frames in which to control the drone (e.g body-up, body-down, local NED, local ENU, downward camera).

To access the partially complete script, `git pull upstream master` in you drone and laptops `aero_control` repository that is part of your catkin workspace. You should now find the file `aero_control/open_loop_control/src/translation_control.py`. Complete portions of code marked as `TODO`.

## Step 2: Setup for Test Flight

Once `translation_control.py` has been completed and pulled onto the drone, run the flight test by following these procedures

- Power on remote control
- Power on the Intel RTF
- Connect to the UAV via QGroundControl
- SSH into the UAV (you will need to have at least four terminals SSHed into the UAV)
- In terminal 1, start distance sensor
	```
    sudo systemctl start aero-teraranger.service
	```
- In terminal 1, start ROS
	```
	roscore
	```

- In terminal 2, start _patched_ optical flow + down-camera streaming. Note: this is different from _standard_ optical flow service that prevents streaming downward-facing camera. Needed for line detection. [See here for more info on setting up](https://github.mit.edu/ma23705/aero_downward_ros/blob/master/README.md)
	```
	cd ~/bwsi-uav/catkin_ws/src/aero-optical-flow/build
	sudo -E ./aero-optical-flow
	```

- Without arming drone, switch to `POSITION CONTROL` mode to ensure previous steps worked as expect. If QGroundControl declares that that position control is rejected, restart process.

- In terminal 3, launch mavros and translation control
	```
	cd ~/bwsi-uav/catkin_ws
	source devel/setup.bash
	roslaunch aero_control translation_control.launch

## Step 2: Setup for Test Flight

Once `translation_control.py` has been completed and pulled onto the drone, run the flight test by following these procedures

- Power on remote control
- Power on the Intel RTF
- Connect to the UAV via QGroundControl
- SSH into the UAV (you will need to have at least four terminals SSHed into the UAV)
- In terminal 1, start distance sensor
	```
    sudo systemctl start aero-teraranger.service
	```
- In terminal 1, start ROS
	```
	roscore
	```

- In terminal 2, start _patched_ optical flow + down-camera streaming. Note: this is different from _standard_ optical flow service that prevents streaming downward-facing camera. Needed for line detection. [See here for more info on setting up](https://github.mit.edu/ma23705/aero_downward_ros/blob/master/README.md)
	```
	cd ~/bwsi-uav/catkin_ws/src/aero-optical-flow/build
	sudo -E ./aero-optical-flow
	```

- Without arming drone, switch to `POSITION CONTROL` mode to ensure previous steps worked as expect. If QGroundControl declares that that position control is rejected, restart process.

- In terminal 3, launch mavros and translation control
	```
	cd ~/bwsi-uav/catkin_ws
	source devel/setup.bash
	roslaunch aero_control translation_control.launch
	```
    

    
## Step 3: Test Flight

<div class="alert alert-warning">

**WARNING:** 

Pilot-in-Command must always be ready to regain control by switching back to `POSCTL` mode. Never take your hands off the remote control!

</div>


In `POSITION CONTROL` mode 

1. Arm the quadrotor
2. Takeoff
3. Position in a safe location in the air
4. Switch to `OFFBOARD` mode to run test
5. Regain control with `POSITION` mode
6. Land quadrotor
7. Disarm
8. Collect [flight log](flight_log_analysis.html)


## Step 4: Coordinate Frame Practice

